In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import dropbox

In [15]:
#Testing with 2010 Bronx data
f = open('2010/2010_bronx.csv','r')
df = pd.read_csv(f)
f.close()

In [20]:
df.count()

Unnamed: 0                        4755
BOROUGH                           4755
NEIGHBORHOOD                      4755
BUILDING CLASS CATEGORY           4755
TAX CLASS AT PRESENT              4755
BLOCK                             4755
LOT                               4755
EASE-MENT                         4755
BUILDING CLASS AT PRESENT         4755
ADDRESS                           4755
APARTMENT NUMBER                  4755
ZIP CODE                          4755
RESIDENTIAL UNITS                 4755
COMMERCIAL UNITS                  4755
TOTAL UNITS                       4755
LAND SQUARE FEET                  4755
GROSS SQUARE FEET                 4755
YEAR BUILT                        4755
TAX CLASS AT TIME OF SALE         4755
BUILDING CLASS AT TIME OF SALE    4755
SALE PRICE                        4755
SALE DATE                         4755
dtype: int64

In [24]:
df2 = df[df['SALE PRICE'] > 0]

In [25]:
df2[df]

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
1,1,2,BATHGATE,01 ONE FAMILY HOMES,1,3035,2,,S1,441 EAST 178 STREET,...,1,1,2,1287,2378,1899,1,S1,116000,2010-07-01
2,2,2,BATHGATE,01 ONE FAMILY HOMES,1,3037,42,,A1,4428 PARK AVENUE,...,1,0,1,3525,1340,1899,1,A1,287000,2010-12-10
3,3,2,BATHGATE,01 ONE FAMILY HOMES,1,3039,64,,A1,467 EAST 185 STREET,...,1,0,1,1667,1296,1910,1,A1,150000,2010-04-20
4,4,2,BATHGATE,01 ONE FAMILY HOMES,1,3046,34,,A1,2085 BATHGATE AVENUE,...,1,0,1,2060,1629,1899,1,A1,145915,2010-07-27
5,5,2,BATHGATE,01 ONE FAMILY HOMES,1,3046,42,,A1,2069 BATHGATE AVENUE,...,1,0,1,1964,1424,1899,1,A1,72790,2010-06-03
6,6,2,BATHGATE,01 ONE FAMILY HOMES,1,3048,18,,A9,2184 BATHGATE AVENUE,...,1,0,1,1768,1236,1930,1,A9,313300,2010-04-09
7,7,2,BATHGATE,01 ONE FAMILY HOMES,1,3048,19,,A5,2186 BATHGATE AVENUE,...,1,0,1,1768,1188,1901,1,A5,299000,2010-06-30
8,8,2,BATHGATE,01 ONE FAMILY HOMES,1,3048,19,,A5,2186 BATHGATE AVENUE,...,1,0,1,1768,1188,1901,1,A5,132000,2010-02-23
9,9,2,BATHGATE,01 ONE FAMILY HOMES,1,3048,28,,A1,540 EAST 182 STREET,...,1,0,1,1209,1048,1901,1,A1,272231,2010-10-04
11,11,2,BATHGATE,02 TWO FAMILY HOMES,1,2927,123,,B1,1489 CROTONA PLACE,...,2,0,2,1900,2394,1994,1,B1,10,2010-10-26
